# Tensorboardの使い方

- PyTorchでTensorboardを使うには？
- loss, val_loss, acc, val_accをロギングする
- グラフの可視化
- TODO: 画像や音声を保存する方法

## 参考

- https://github.com/lanpa/tensorboard-pytorch
- https://qiita.com/r9y9/items/d54162d37ec4f110f4b4

In [3]:
!pip install tensorboardX

## ロギングの実験

In [5]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorboardX import SummaryWriter

cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')

# Hyperparameters 
num_epochs = 50
batch_size = 128
learning_rate = 0.01

# load data
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# model
class MultiLayerPerceptron(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.fc2(out)
        return out

model = MultiLayerPerceptron(784, 512, 10)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train(train_loader):
    model.train()

    running_loss = 0
    correct = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28 * 28)
        if cuda:
            images, labels = images.cuda(), labels.cuda()
        images = Variable(images)
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        running_loss += loss.data[0]

        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels.data).sum()

    train_loss = running_loss / len(train_loader)
    train_acc = correct / len(train_loader.dataset)

    return train_loss, train_acc


def valid(test_loader):
    model.eval()

    running_loss = 0
    correct = 0

    for batch_idx, (images, labels) in enumerate(test_loader):
        images = images.view(-1, 28 * 28)
        if cuda:
            images, labels = images.cuda(), labels.cuda()
        images = Variable(images, volatile=True)
        labels = Variable(labels, volatile=True)

        outputs = model(images)

        loss = criterion(outputs, labels)
        running_loss += loss.data[0]

        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels.data).sum()

    val_loss = running_loss / len(test_loader)
    val_acc = correct / len(test_loader.dataset)
    
    return val_loss, val_acc

# training loop
writer = SummaryWriter()
max_acc = 0

for epoch in range(num_epochs):
    loss, acc = train(train_loader)
    val_loss, val_acc = valid(test_loader)

    # logging
    writer.add_scalar('loss', loss, epoch)
    writer.add_scalar('acc', acc, epoch)
    writer.add_scalar('val_loss', val_loss, epoch)
    writer.add_scalar('val_acc', val_acc, epoch)

    print('epoch %d, loss: %.4f acc: %.4f val_loss: %.4f val_acc: %.4f'
          % (epoch, loss, acc, val_loss, val_acc))

    # save the best model
    if val_acc > max_acc:
        print('val_acc improved from %.5f to %.5f!' % (max_acc, val_acc))
        max_acc = val_acc
        model_file = 'epoch%03d-%.4f-%.4f.pth' % (epoch, val_loss, val_acc)
        torch.save(model.state_dict(), model_file)

writer.export_scalars_to_json('all_scalars.json')
writer.close()

epoch 0, loss: 1.6287 acc: 0.6804 val_loss: 0.9516 val_acc: 0.8274
val_acc improved from 0.00000 to 0.82740!
epoch 1, loss: 0.7303 acc: 0.8462 val_loss: 0.5600 val_acc: 0.8709
val_acc improved from 0.82740 to 0.87090!
epoch 2, loss: 0.5170 acc: 0.8726 val_loss: 0.4460 val_acc: 0.8886
val_acc improved from 0.87090 to 0.88860!
epoch 3, loss: 0.4385 acc: 0.8853 val_loss: 0.3942 val_acc: 0.8966
val_acc improved from 0.88860 to 0.89660!
epoch 4, loss: 0.3973 acc: 0.8923 val_loss: 0.3629 val_acc: 0.8999
val_acc improved from 0.89660 to 0.89990!
epoch 5, loss: 0.3710 acc: 0.8978 val_loss: 0.3428 val_acc: 0.9050
val_acc improved from 0.89990 to 0.90500!
epoch 6, loss: 0.3522 acc: 0.9015 val_loss: 0.3273 val_acc: 0.9094
val_acc improved from 0.90500 to 0.90940!
epoch 7, loss: 0.3377 acc: 0.9058 val_loss: 0.3145 val_acc: 0.9116
val_acc improved from 0.90940 to 0.91160!
epoch 8, loss: 0.3259 acc: 0.9082 val_loss: 0.3050 val_acc: 0.9142
val_acc improved from 0.91160 to 0.91420!
epoch 9, loss: 0.31

In [7]:
!tensorboard --logdir runs

/Users/koichiro.mori/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.5.1 at http://o-05349-mac.local:6006 (Press CTRL+C to quit)
^C
